In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
#papers_df is pandas dataframe object
papers_df = pd.read_json('NIP_DataSet/papers.json',lines = True)
papers_df.rename(columns = {"id" : "paper_id"},inplace=True)
papers_df.head()

ValueError: Unexpected character found when decoding 'NaN'

In [ ]:
authors_df = pd.read_json('NIP_DataSet/authors.json',lines = True)
authors_df.rename(columns = {"id" : "author_id"},inplace=True)
authors_df.head()

In [ ]:
paper_authors_df = pd.read_json('NIP_DataSet/paper_authors.json',lines = True)

paper_authors_df.head()

In [ ]:
print("Papers : ", papers_df.shape)
print("Authors : ", authors_df.shape)
print("Paper with Authors : ", paper_authors_df.shape)

In [ ]:
merge_df = pd.merge(paper_authors_df,authors_df, on='author_id')
merge_df.head(10)

In [ ]:
merge_df = pd.merge(merge_df,papers_df, on='paper_id')
merge_df.head(10)

In [ ]:
merge_df.info()

In [ ]:
merge_df = merge_df.drop(columns = ['event_type', 'pdf_name', 'abstract'], axis=1)
merge_df.head(10)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')

merge_df['paper_text'] = merge_df['paper_text'].fillna('')


In [ ]:
tfv_matrix = tfv.fit_transform(merge_df['paper_text'])

In [ ]:
tfv_matrix.shape

In [ ]:
def GetPaperByUserID(userID,N):
    userRatings = merge_df[merge_df["author_id"]==userID]
    sortedRatings = pd.DataFrame.sort_values(userRatings,['year'],ascending=[0])[:N] )
    return sortedRatings

In [ ]:
GetPaperByUserID(userID,N)

In [ ]:
from sklearn.metrics.pairwise import sigmoid_kernel

# Compute the sigmoid kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

In [ ]:
sig[0]

In [ ]:
merge_df.index

In [ ]:
# Reverse mapping of indices and paper titles
indices = pd.Series(merge_df.index, index = merge_df['paper_id']).drop_duplicates()

In [ ]:
indices[157]

In [ ]:
list(enumerate(sig[157]))

In [ ]:
sorted(list(enumerate(sig[indices[157]])), key=lambda x: x[:], reverse=True)

In [ ]:
def give_rec(title, sig=sig):
    # Get the index corresponding to title
    idx = indices[title]

    print(idx)
    # Get the pairwsie similarity scores 
    sig_scores = list(enumerate(sig[idx]))

    # Sort the paper 
    sig_scores = sorted(sig_scores, key=lambda x: x[:], reverse=True)

    # Scores of the 10 most similar papers
    sig_scores = sig_scores[1:11]

    # Paper indices
    paper_indices = [i[0] for i in sig_scores]

    # Top 10 most similar papers
    return merge_df['paper_id'].iloc[paper_indices]

In [ ]:
# Testing our content-based recommendation system with Genetic Algorithms and its use with back-propagation network
give_rec(157)

In [ ]:
merge_df.iloc[4]